### 1. Воздушные перевозки

Скачайте и распакуте следующий [архив](http://stat-computing.org/dataexpo/2009/2008.csv.bz2). Описание полей таблицы дано [здесь](http://www.transtats.bts.gov/Fields.asp?Table_ID=236).

Данные должны содержать перелеты воздушных судов в 2008 году (вероятно в США)

In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np
import scipy.optimize as opt
import sympy
import pandas as pd
plt.style.use('ggplot')

%matplotlib inline

**Задание 1:** Рассчитать топ-10 массовых перевозчиков.

In [170]:
df1 = pd.read_csv('2008.csv')



In [300]:
print(df1.size)
print(df1.shape)
print(df1.columns.values)
print(df1.index.values)

203282112
(7009728, 29)
['Year' 'Month' 'DayofMonth' 'DayOfWeek' 'DepTime' 'CRSDepTime' 'ArrTime'
 'CRSArrTime' 'UniqueCarrier' 'FlightNum' 'TailNum' 'ActualElapsedTime'
 'CRSElapsedTime' 'AirTime' 'ArrDelay' 'DepDelay' 'Origin' 'Dest'
 'Distance' 'TaxiIn' 'TaxiOut' 'Cancelled' 'CancellationCode' 'Diverted'
 'CarrierDelay' 'WeatherDelay' 'NASDelay' 'SecurityDelay'
 'LateAircraftDelay']
[      0       1       2 ..., 7009725 7009726 7009727]


In [327]:
car = df1['UniqueCarrier'].value_counts(ascending = False).head(10)
carrier = pd.DataFrame({'Airplane code' : car.keys(),
                   'Number of Flights' : car.values}, index = [x for x in range (1,11)])
carrier

,Airplane code,Number of Flights
1,WN,1201754
2,AA,604885
3,OO,567159
4,MQ,490693
5,US,453589
6,DL,451931
7,UA,449515
8,XE,374510
9,NW,347652
10,CO,298455


In [302]:
cc = df1['CancellationCode'].dropna(how='any').value_counts()
concell = pd.DataFrame({'Number' : cc.values,'Reason' : cc.keys()}, index = [x for x in range(1,5)])
concell

,Number,Reason
1,54904,B
2,54330,A
3,28188,C
4,12,D


Взяв нужный столбец, убрали все рейсы, в которых не было отмены.

**Задание 3:** Найти самый популярный маршрут, выдать статистику по данному маршруту.

In [303]:
df1_pop = df1.groupby(['Origin','Dest']).size().sort_values(ascending = False).iloc[:1]
df1_pop

Origin  Dest
SFO     LAX     13788
dtype: int64

In [304]:
df1[(df1['Origin'] == 'SFO') & (df1['Dest'] == 'LAX')].describe()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,...,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,13788,13788.000000,13788.000000,13788.000000,13299.000000,13788.000000,13286.000000,13788.000000,13788.000000,13285.000000,...,13788,13286.000000,13299.000000,13788.000000,13788.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000
mean,2008,6.337322,15.730418,3.862997,1371.032408,1356.596388,1481.378820,1493.550116,1862.825573,82.417840,...,337,9.912163,17.273554,0.035466,0.001015,9.689688,0.360280,7.630808,0.026416,42.157225
std,0,3.358147,8.791381,1.973866,489.328348,472.940974,502.009856,473.465680,1999.167552,10.184247,...,0,5.120369,7.126813,0.184960,0.031850,27.959199,8.216006,16.685259,1.009160,46.774688
min,2008,1.000000,1.000000,1.000000,3.000000,540.000000,1.000000,5.000000,4.000000,58.000000,...,337,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2008,3.000000,8.000000,2.000000,941.000000,945.000000,1054.000000,1110.000000,817.000000,75.000000,...,337,7.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
50%,2008,6.000000,16.000000,4.000000,1355.000000,1315.000000,1507.000000,1443.000000,1153.000000,81.000000,...,337,9.000000,16.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,28.000000
75%,2008,9.000000,23.000000,5.000000,1809.000000,1756.000000,1919.000000,1920.000000,1957.000000,88.000000,...,337,12.000000,20.000000,0.000000,0.000000,7.000000,0.000000,11.000000,0.000000,64.000000
max,2008,12.000000,31.000000,7.000000,2400.000000,2240.000000,2400.000000,2359.000000,7815.000000,196.000000,...,337,106.000000,84.000000,1.000000,1.000000,339.000000,406.000000,401.000000,54.000000,373.000000


**Задание 4:** Найти топ-5 рейсов по каждому из типов delay. Каким перевозчикам они принадлежат?

По каждому из типов задержек сделали сортировку, вывели первые пять задержек по времени.

In [260]:
print('Carrier delay', df1['CarrierDelay'].sort_values(ascending = False).head(5), sep = '\n', end = '\n\n')
print('WeatherDelay', df1['WeatherDelay'].sort_values(ascending = False).head(5), sep = '\n', end = '\n\n')
print('NASDelay', df1['NASDelay'].sort_values(ascending = False).head(5), sep = '\n', end = '\n\n')
print('SecurityDelay', df1['SecurityDelay'].sort_values(ascending = False).head(5), sep = '\n', end = '\n\n')
print('LateAircraftDelay', df1['LateAircraftDelay'].sort_values(ascending = False).head(5), sep = '\n')


Carrier delay
2235378    2436
2832617    1951
3387883    1707
5232546    1552
2232494    1542
Name: CarrierDelay, dtype: float64

WeatherDelay
6842014    1352
6868167    1297
6932676    1225
6916589    1153
1098428    1148
Name: WeatherDelay, dtype: float64

NASDelay
503727     1357
4117650    1337
1705852    1289
2189463    1207
1030111    1195
Name: NASDelay, dtype: float64

SecurityDelay
2116823    392
5231914    357
3114684    284
3100409    280
6575811    254
Name: SecurityDelay, dtype: float64

LateAircraftDelay
4099781    1316
5039477    1303
1426738    1254
6308375    1236
1620103    1184
Name: LateAircraftDelay, dtype: float64


**Задание 5:** Определить, как распределено количество рейсов от времени дня?

По столбцу времени отправления сделали 4 разделения на периоды дняб затем посчитали количество вылетов для каждого периода и получили следующие выводы:

In [261]:
print('Morning', df1['DepTime'][(df1['DepTime'] > 600) & (df1['DepTime'] < 1200)].count(), sep = '\n', end = '\n\n')
print('Day', df1['DepTime'][(df1['DepTime'] > 1200) & (df1['DepTime'] < 1800)].count(), sep = '\n', end = '\n\n')
print('Evening', df1['DepTime'][(df1['DepTime'] > 1800) & (df1['DepTime'] < 2400)].count(), sep = '\n', end = '\n\n')
print('Night', df1['DepTime'][(df1['DepTime'] > 0) & (df1['DepTime'] < 600)].count(), sep = '\n')


Morning
2627237

Day
2544966

Evening
1485283

Night
179949


**Задание 6:** Определить "сезонность" во временных рядах по количеству вылетов на каждый день.

### 2. Продолжаем изучать Pandas

Ввиду ограниченности времени, мы не успеваем рассмотреть всё на семинарах. Поэтому часть работы ложится на вас..

Загрузите две csv таблицы [отсюда](https://github.com/emorisse/FBI-Hate-Crime-Statistics) и импортируйте через Pandas. В них содержатся агрегированные данные по различным организациям\городам США, содержащие количество обращений по тем или иным случаям нарушения закона. В одной таблице (table13.csv) перечислены управления, которые сообщали об этих случаях, в другой - которые не сообщали.

**Задание 1:** Изучите документации для методов `unique_values`, `isin`. С помощью этой функции, определите, содержатся ли в двух таблицах одни и те же штаты.

In [139]:
df_tab1 = pd.read_csv('table13.csv')
df_tab2 = pd.read_csv('table14.csv')

In [313]:
print(df_tab1.shape)
print(df_tab1.size)
print(df_tab1.columns.values)
print(df_tab1.index.values)


(1826, 15)
27390
['State' 'Agency type' 'Agency name' 'Race' 'Religion' 'Sexual orientation'
 'Ethnicity' 'Disability' 'Gender' 'Gender Identity' '1st quarter'
 '2nd quarter' '3rd quarter' '4th quarter' 'Population']
[   0    1    2 ..., 1823 1824 1825]


In [314]:
print(df_tab2.shape)
print(df_tab2.size)
print(df_tab2.columns.values)
print(df_tab2.index.values)


(12826, 8)
102608
['State' 'Agency Type' 'Agency Name' 'Q1' 'Q2' 'Q3' 'Q4' 'Population']
[    0     1     2 ..., 12823 12824 12825]


In [318]:
print('Tab1 in Tab2\'s elements', df_tab1['State'][df_tab1['State'].isin(df_tab2['State']) == True].count(), sep = '\n', end = '\n\n')
print('Tab2 in Tab1\'s elements', df_tab2['State'][df_tab2['State'].isin(df_tab1['State']) == True].count(), sep = '\n')

Tab1 in Tab2's elements
1819

Tab2 in Tab1's elements
12826


Второе значение полностью совпадает с количеством штатов во второй таблице, значит, все штаты 2 таблицы содержатся в 1 табл., но не все штаты 1 табл. есть во 2табл.

**Задание 2:** Изучите документации для методов `append`, `merge` и `concat`. Совместите две таблицы в одну.

In [316]:
app_1 = df_tab2.append(df_tab1, ignore_index = True)
app_1

,1st quarter,2nd quarter,3rd quarter,4th quarter,Agency Name,Agency Type,Agency name,Agency type,Disability,Ethnicity,...,Gender Identity,Population,Q1,Q2,Q3,Q4,Race,Religion,Sexual orientation,State
0,NaN,NaN,NaN,NaN,Ashford,Cities,NaN,NaN,NaN,NaN,...,NaN,"2,177",0,0,0,0,NaN,NaN,NaN,Alabama
1,NaN,NaN,NaN,NaN,Ashland,Cities,NaN,NaN,NaN,NaN,...,NaN,"1,926",0,0,0,0,NaN,NaN,NaN,Alabama
2,NaN,NaN,NaN,NaN,Auburn,Cities,NaN,NaN,NaN,NaN,...,NaN,"57,970",0,0,0,0,NaN,NaN,NaN,Alabama
3,NaN,NaN,NaN,NaN,Berry,Cities,NaN,NaN,NaN,NaN,...,NaN,"1,124",NaN,0,NaN,NaN,NaN,NaN,NaN,Alabama
4,NaN,NaN,NaN,NaN,Childersburg,Cities,NaN,NaN,NaN,NaN,...,NaN,"5,111",0,0,0,0,NaN,NaN,NaN,Alabama
5,NaN,NaN,NaN,NaN,Courtland,Cities,NaN,NaN,NaN,NaN,...,NaN,607,0,0,0,0,NaN,NaN,NaN,Alabama
6,NaN,NaN,NaN,NaN,Demopolis,Cities,NaN,NaN,NaN,NaN,...,NaN,"7,240",0,0,0,0,NaN,NaN,NaN,Alabama
7,NaN,NaN,NaN,NaN,Eclectic,Cities,NaN,NaN,NaN,NaN,...,NaN,"1,015",0,0,0,0,NaN,NaN,NaN,Alabama
8,NaN,NaN,NaN,NaN,Enterprise,Cities,NaN,NaN,NaN,NaN,...,NaN,"28,145",0,0,0,0,NaN,NaN,NaN,Alabama
9,NaN,NaN,NaN,NaN,Eutaw,Cities,NaN,NaN,NaN,NaN,...,NaN,"2,871",0,0,0,0,NaN,NaN,NaN,Alabama


Создали новую таблицу, присоединив вторую к первой. Так как индексы не имеют значения, можем ввести общее индексирование для соединенных таблиц.

**Задание 3:** Изучите документации для методов `drop_dublicates` и `dropna`. Используйте метод `dropna` для очистки большой таблицы от пропусков. Что произошло? Почему? Исправьте таблицы, чтобы получить корректный ответ.

In [131]:
app_1.dropna()

,1st quarter,2nd quarter,3rd quarter,4th quarter,Agency Name,Agency Type,Agency name,Agency type,Disability,Ethnicity,...,Gender Identity,Population,Q1,Q2,Q3,Q4,Race,Religion,Sexual orientation,State


Воспользовавшись dropna, получили пустую таблицу. Это произошло из-за того, что названия колонок исходных таблиц не совпадают, т.е. при совмещении таблиц новые колонки добавлялись, а не объединялись, поэтому они все наполовину пустые. Один из способов решения - объединить собственноручно столбцы, которые по смыслу содержат одинаковую информацию, но отличаются в названии, переименовав их. Затем объединим и используем dropna().

In [279]:
df_tab1re = df_tab1.rename(columns = {'Agency type':'Agency Type', 'Agency name':'Agency Name', '1st quarter':'Q1',
 '2nd quarter':'Q2', '3rd quarter':'Q3', '4th quarter':'Q4'})
df_tab1re

,State,Agency Type,Agency Name,Race,Religion,Sexual orientation,Ethnicity,Disability,Gender,Gender Identity,Q1,Q2,Q3,Q4,Population
0,Alabama,Cities,Florence,2,0,0,0,0,0,0,0,1,0,1,"39,481"
1,Alabama,Cities,Hoover,0,0,1,0,0,0,0,0,1,0,0,"84,139"
2,Alabama,Cities,Prattville,2,0,0,0,0,0,0,1,0,1,0,"35,154"
3,Alabama,Cities,Tuscaloosa,1,0,0,0,0,0,0,0,0,1,0,"94,126"
4,Alaska,Cities,Anchorage,8,0,0,0,0,0,0,2,3,3,0,"299,455"
5,Arizona,Cities,Apache Junction,1,0,0,0,0,0,0,1,0,0,NaN,"36,626"
6,Arizona,Cities,Avondale,4,0,0,1,0,0,0,1,1,3,0,"78,905"
7,Arizona,Cities,Eagar,0,0,0,1,0,0,0,0,1,0,0,"5,034"
8,Arizona,Cities,El Mirage,1,0,0,0,0,0,0,0,0,1,0,"32,837"
9,Arizona,Cities,Gilbert,1,0,0,0,0,0,0,1,0,0,0,"225,232"


In [281]:
app_2 = pd.concat([df_tab1re, df_tab2])
app_2.dropna()

,Agency Name,Agency Type,Disability,Ethnicity,Gender,Gender Identity,Population,Q1,Q2,Q3,Q4,Race,Religion,Sexual orientation,State
0,Florence,Cities,0,0,0,0,"39,481",0,1,0,1,2,0,0,Alabama
1,Hoover,Cities,0,0,0,0,"84,139",0,1,0,0,0,0,1,Alabama
2,Prattville,Cities,0,0,0,0,"35,154",1,0,1,0,2,0,0,Alabama
3,Tuscaloosa,Cities,0,0,0,0,"94,126",0,0,1,0,1,0,0,Alabama
4,Anchorage,Cities,0,0,0,0,"299,455",2,3,3,0,8,0,0,Alaska
6,Avondale,Cities,0,1,0,0,"78,905",1,1,3,0,4,0,0,Arizona
7,Eagar,Cities,0,1,0,0,"5,034",0,1,0,0,0,0,0,Arizona
8,El Mirage,Cities,0,0,0,0,"32,837",0,0,1,0,1,0,0,Arizona
9,Gilbert,Cities,0,0,0,0,"225,232",1,0,0,0,1,0,0,Arizona
10,Glendale,Cities,0,1,0,0,"234,006",2,0,3,2,2,3,1,Arizona


**Задание 4:** Изучите документации для методов `set_index`, `reset_index` и `pivot_table`. Проиндексируйте общую таблицу по переменным State и Agency Type (одновременно). У вас должен появиться *мультииндекс*.

In [285]:
setidx = app_2.set_index(['State','Agency Type'])
setidx

Agency Name  \
State    Agency Type                                                              
Alabama  Cities                                                        Florence   
         Cities                                                          Hoover   
         Cities                                                      Prattville   
         Cities                                                      Tuscaloosa   
Alaska   Cities                                                       Anchorage   
Arizona  Cities                                                 Apache Junction   
         Cities                                                        Avondale   
         Cities                                                           Eagar   
         Cities                                                       El Mirage   
         Cities                                                         Gilbert   
         Cities                                                        Glendale   
         Cities                                                        Goodyear   
         Cities                                                        Maricopa   
         Cities                                                            Mesa   
         Cities                                                         Phoenix   
         Cities                                                        Prescott   
         Cities                                                      Scottsdale   
         Cities                                                           Tempe   
         Cities                                                          Tucson   
         Cities                                                            Yuma   
         Universities and Colleges                  Northern Arizona University   
         Universities and Colleges                        University of Arizona   
         Metropolitan Counties                                          Cochise   
         Metropolitan Counties                                         Maricopa   
         Metropolitan Counties                                             Pima   
         Metropolitan Counties                                             Yuma   
Arkansas Cities                                                     Bentonville   
         Cities                                                      Berryville   
         Cities                                                           Cabot   
         Cities                                                         England   
...                                                                         ...   
Wyoming  Cities                                                        Sundance   
         Cities                                                     Thermopolis   
         Cities                                                      Torrington   
         Cities                                                       Wheatland   
         Cities                                                         Worland   
         Universites and Colleges   Northern Wyoming Community College District   
         Universites and Colleges                         University of Wyoming   
         Metropolitan Counties                                          Laramie   
         Metropolitan Counties                                          Natrona   
         Nonmetropolitan Counties                                        Albany   
         Nonmetropolitan Counties                                      Big Horn   
         Nonmetropolitan Counties                                      Campbell   
         Nonmetropolitan Counties                                        Carbon   
         Nonmetropolitan Counties                                      Converse   
         Nonmetropolitan Counties                                         Crook   
         Nonmetropolitan Counties                                       Fremont   
 

**Задание 5:** В Pandas можно идерировать по DataFrame! Ищучите документации для методов `iteritems`, `iterrows`, а так же посмотрите, как можно итерировать по сгруппированным DataFrame'ам. Выделите подтаблицу, состаящую из данных по 3 произвольным штатам и сгруппируйте её по переменным State и Agency Type. С помощью изученных методов (то есть итеративно) выведите суммарную популяцию по каждой паре <Штат>-<Тип>

In [345]:
newtab = setidx.loc[(setidx.index.get_level_values('State') == 'Texas'),
                   (setidx.index.get_level_values('State') == 'Colorado'),
                    (setidx.index.get_level_values('State') == 'California')]

newtab

KeyError: 'Key length (3) exceeds index depth (2)'

## 3. Метод градиентного спуска

Напомним суть метода градиентого спуска в контексте задачи линейной регрессии.

Дано описание $n$ объектов по $m$ признакам. Обычно оно выражается в виде матрицы размера $n \times m$: $X = [x^{(i)}_j]^{i=1\dots n}_{j=1\dots m} $.<br\> ($x^{(i)}_j$ означает $j$-ый признак $i$-го объекта) <br\>
Дана зависимая переменная, которая тоже имеет отношение к этим объекам: $y$ - вектор длины $n$.

Наша задача, выявить **линейную** зависимость между признаками в $X$ и значениями в $y$:
$$\hat{y} = X\beta \quad \Leftrightarrow \quad \hat{y}^{(i)} = \beta_0 + \beta_1x^{(i)}_1 + \dots$$

И сделать это так, чтобы квадрат суммы ошибок наших оценок был минимален:
$$ L(\beta) = \frac{1}{2n}(\hat{y} - y)^{\top}(\hat{y} - y) = \frac{1}{2n}(X\beta - y)^{\top}(X\beta - y) \rightarrow \min$$ $$ \Updownarrow $$  $$ L(\beta_0,\beta_1,\dots) = \frac{1}{2n}\sum^{n}_{i=1}(\hat{y}^{(i)} - y^{(i)})^2 = \frac{1}{2n}\sum^{n}_{i=1}(\beta_0 + \beta_1x^{(i)}_1 + \dots - y^{(i)})^2  \rightarrow \min $$

Значение в $X$ и $y$ нам даны. Нам неизвестны только значения коэффициентов $\beta$.<br\> Соответственно, нужно найти такие значения $\beta$, что функция $L(\beta) \rightarrow \min.$

На семинаре мы выяснили, чему равен градиент функции потерь $L(\beta_0, \beta_1):$
$$ \frac{\partial L}{\partial \beta_0} = \frac{1}{n}\sum^{n}_{i=1}(\beta_0 + \beta_1x^{(i)}_1 - y^{(i)})$$
$$ \frac{\partial L}{\partial \beta_1} = \frac{1}{n}\sum^{n}_{i=1}(\beta_0 + \beta_1x^{(i)}_1 - y^{(i)})x_1^{(i)}$$

Иногда проще это записать в виде матриц:
$$ \frac{\partial L}{\partial \beta} = X^\top(X\beta - y)$$


Метод градиентного спуска заключается в итеративном и **одновременном(!!!)** обновлении значений $\beta$ в направлении, противоположному градиенту:
$$ \beta := \beta - \alpha\frac{\partial L}{\partial \beta}$$

**Задача 1:** Загрузите [данные](http://bit.ly/1gIQs6C) по характеристикам автомобилей Honda Accord. Названия столбцов говорят сами за себя.

Составьте матрицу $X$, состоящую из единичного столбца и признака "пробег". <br\>
Составьте вектор $y$ состоящий из стоимости автомобиля.

Если для импорта вы используете `pandas`, то через .values из DataFrame можно получить numpy array.

In [137]:
df = pd.read_csv("accord_sedan_training.csv")

**Задача 2:** Изобразите функционал квадратичной ошибки $L(\beta_0, \beta_1)$ в 3D и его линии уровня

In [18]:
# Здесь ваш код

**Задача 3:** Напишите код, реализующий метод градиентного спуска. На вход метод должен принимать исходные данные, максимальное кол-во итераций и коэффициент скорости спуска.

In [19]:
# Здесь ваш код

**Задача 4:** Выведите полученные значения коэфициентов $\beta$. <br\>Как они интерпретируются?  <br\>Изобразите на одном графике $[y - \text{"пробег"}]$ точки и полученную прямую.

In [20]:
# Здесь ваш код